<a href="https://colab.research.google.com/github/LimHyeongSoo/Samsung_Securities/blob/main/Samsung_Securities/HyeongSoo/Kakao_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install transformers[torch] accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

## Train_1 => Dataset : KB 은행

In [ ]:
import pandas as pd

# CSV 파일 로드 및 첫 몇 줄 확인
data = pd.read_csv("/content/drive/MyDrive/삼성증권/Data/financial_mrc_test.csv")
print(data.head())

                                            Question   Answer  \
0  위성호 은행장에 의해 개편된 신설조직 중, ‘원 신한(One Shinhan)’ 관점...    GIB그룹   
1  ‘초(超)격차의 리딩뱅크’와 ‘World Class Bank 신한’의 꿈을 실현하기...  위성호 은행장   
2  신용보증기금이 유망 스타트업의 민간투자 유치 기회를 확대하고 크라우드펀딩 성공기업에...      와디즈   
3  스타트업 네스트 신청 시 1차 서류심사가 면제될 펀딩 성공기업의 기준 금액은 몇 억...      1억원   
4  신한금융그룹이 후원하고, 장애청년들의 아름다운 도전을 응원하는 ‘장애청년드림팀’이 ...  한국프레스센터   

                                             Context  
0  (금융경제신문 문혜원 기자)신한은행은 지난 6일 ▲본부부서 조직개편과 ▲현장 영업동...  
1  (금융경제신문 문혜원 기자)신한은행은 지난 6일 ▲본부부서 조직개편과 ▲현장 영업동...  
2  (금융경제신문 문혜원 기자)신용보증기금이 유망 스타트업의 민간투자 유치 기회를 확대...  
3  (금융경제신문 문혜원 기자)신용보증기금이 유망 스타트업의 민간투자 유치 기회를 확대...  
4  (금융경제신문 문혜원 기자)신한금융그룹이 후원하고, 장애청년들의 아름다운 도전을 응...  


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict


# 데이터 전처리
data = data.rename(columns={"Question": "question", "Context": "context", "Answer": "answer_text"})
data['answers'] = data.apply(lambda row: {"text": [row['answer_text']], "answer_start": [row['context'].find(row['answer_text'])]}, axis=1)
data = data[['question', 'context', 'answers']]

# 데이터셋 준비
dataset = Dataset.from_pandas(data)
datasets = DatasetDict({"train": dataset, "validation": dataset})

# 토크나이저 및 모델 불러오기
model_name = "kakaobank/kf-deberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # If the answer is out of the span (in the truncated part), label it as (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx < context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_start
            while idx < context_end and offset[idx][1] < end_char:
                idx += 1
            end_positions.append(idx)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/746M [00:00<?, ?B/s]

Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at kakaobank/kf-deberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

In [ ]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# 모델 훈련
trainer.train()

# 훈련된 모델 저장
model.save_pretrained("/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1")
tokenizer.save_pretrained("/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1")


Epoch,Training Loss,Validation Loss
1,No log,3.309202
2,No log,1.420393
3,No log,1.062933


('/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/tokenizer_config.json',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/special_tokens_map.json',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/vocab.txt',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/added_tokens.json',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/tokenizer.json')

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

# 저장된 모델과 토크나이저 로드
model_path = "/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

# pipeline 설정
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)

# 예제 질문과 컨텍스트
question = "위성호 은행장에 의해 개편된 신설조직 중, ‘원 신한(One Shinhan)’ 관점의 그룹경쟁력 강화를 위한 그룹 매트릭스 조직으로 사업부문 소속 직원들의 협업을 통해 역량을 집중할 계획을 가진 그룹명은?"
context = '''(금융경제신문 문혜원 기자)신한은행은 지난 6일 ▲본부부서 조직개편과 ▲현장 영업동력 강화 및 커뮤니티 지원에 중점을 둔 하반기 정기인사를 실시했다고 밝혔다.
이번 조직개편은 디지털과 글로벌을 향한 신한만의 새로운 길을 만들어 ‘초(超)격차의 리딩뱅크’와 ‘World Class Bank 신한’의 꿈을 실현하기 위한 위성호 은행장의 전략을 바탕으로 진행됐다.
조직개편의 주요 내용은 ▲디지털그룹 ▲GIB그룹 ▲대기업그룹 ▲글로벌사업본부 신설이다. 디지털그룹은 기존 분산되어 있던 디지털 인적·물적 역량 및 사업전략을 총괄하는 조직으로 신한은행의 디지털전략을 총괄하는 디지털전략본부, 모바일 채널 통합 플랫폼 구축을 위한 디지털채널본부, 빅데이터 분석역량 강화를 위한 빅데이터센터로 구성된다.
또한 유연한 디지털 조직 운영을 위해 디지털그룹 내 A.I, 블록체인 등 총 7개의 랩(Lab)조직을 신설했다.
GIB그룹과 대기업그룹은 자본시장 경쟁력 강화를 위해 기존 CIB그룹을 분리 확대 개편하면서 신설됐다.
GIB그룹은 ‘원 신한(One Shinhan)’ 관점의 그룹경쟁력 강화를 위한 그룹 매트릭스 조직으로 사업부문 소속 직원들의 협업을 통해 역량을 집중할 계획이다.
글로벌사업본부는 글로벌시장 경쟁력 강화를 위해 글로벌사업그룹 내 신설된 조직으로 앞으로 신한은행의 글로벌영업과 전략을 담당하게 된다.'''

# 질문에 대한 답변 생성
result = question_answerer(question=question, context=context)
print(result)

{'score': 0.09373865276575089, 'start': 447, 'end': 460, 'answer': 'GIB그룹과 대기업그룹은'}


## Train_2 => Dataset : squad_kor_v1

In [ ]:
from datasets import load_dataset
import pandas as pd
# KorQuAD 데이터셋 로드
dataset = load_dataset("squad_kor_v1")

# 데이터셋 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})


In [ ]:
train_df = pd.DataFrame(dataset['train'])

# 데이터셋 확인
train_df.head()

,id,title,context,question,answers
0,6566495-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,"{'text': ['교향곡'], 'answer_start': [54]}"
1,6566495-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,"{'text': ['1악장'], 'answer_start': [421]}"
2,6566495-0-2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,"{'text': ['베토벤의 교향곡 9번'], 'answer_start': [194]}"
3,6566518-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,"{'text': ['파우스트'], 'answer_start': [15]}"
4,6566518-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,"{'text': ['합창교향곡'], 'answer_start': [354]}"


In [ ]:
from datasets import load_dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

# KorQuAD 데이터셋 로드
dataset = load_dataset("squad_kor_v1")

# train 및 validation 데이터셋을 pandas DataFrame으로 변환
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])

# 무작위로 5000개 샘플 추출
train_df_sampled = train_df.sample(n=5000, random_state=42)
validation_df_sampled = validation_df.sample(n=500, random_state=42)

# 샘플링된 데이터셋을 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df_sampled)
validation_dataset = Dataset.from_pandas(validation_df_sampled)

# 모델과 토크나이저 로드
model_dir = "/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForQuestionAnswering.from_pretrained(model_dir)

# 전처리 함수 정의
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx < context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_start
            while idx < context_end and offset[idx][1] < end_char:
                idx += 1
            end_positions.append(idx)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# 데이터셋 전처리
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
)

# 모델 훈련
trainer.train()

# 훈련된 모델 저장
model.save_pretrained("/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1")
tokenizer.save_pretrained("/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.730700,0.661034
2,0.368200,0.743540
3,0.227700,0.920360


('/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/tokenizer_config.json',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/special_tokens_map.json',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/vocab.txt',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/added_tokens.json',
 '/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1/tokenizer.json')

In [ ]:
from transformers import pipeline

# 테스트 데이터 준비
test_df = pd.DataFrame(dataset['validation']).sample(n=10, random_state=42)  # 무작위로 10개 샘플 추출
test_dataset = Dataset.from_pandas(test_df)
print(test_df)

                id              title  \
426    6482359-1-0  김정남_(조선민주주의인민공화국)   
1181   6588960-0-0              원진레이온   
5292   6518409-1-1               송_평공   
3836   6556751-4-1         중화인민공화국의_법   
3053  6459703-15-0                이회창   
4633   6457541-1-0                아가씨   
3822   6196357-1-1         중화인민공화국의_법   
1328   6529200-2-1           마리_앙투아네트   
4479   6464784-5-0                전지현   
2209   6588563-5-1      우루과이_축구_국가대표팀   

                                                context  \
426   2017년 2월 13일 오전 9시경 김정남은 말레이시아 세팡에 위치한 쿠알라룸푸르 ...   
1181  1977년 입사하여 원액2과에서 7년간 근무하고 1983년 퇴사한 김봉환은 1990...   
5292  평공 치세 초기에는 이웃한 정나라가 초나라와 진나라 사이를 오갔고, 송나라는 줄곧 ...   
3836  중국에 있어서 법원(法源)은, 중화인민공화국 헌법을 정점으로 하여, 법률, 행정법규...   
3053  8월 26일 이회창은 심대평 대표에 대한 총리 기용설과 관련, "앞으로 이에 관한 ...   
4633  늦은 밤 히데코의 저택에 도착한 숙희에게 집사 사사키(김해숙)는 숙희가 하녀로 위장...   
3822  아편 전쟁, 태평천국의 난, 아로호 사건이라는 대규모 내우외환에 직명한 청나라 정부...   
1328  프랑스의 다른 왕비들과 비교하면 마리 앙투아네트가 쓴 돈은 그다지 많은 수준이 아니...   
4479  한동안 슬럼프에 빠져

In [ ]:
# 데이터 전처리
def preprocess_test_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    return inputs

tokenized_test_dataset = test_dataset.map(preprocess_test_function, batched=True, remove_columns=test_dataset.column_names)


SyntaxError: invalid syntax (<ipython-input-21-505f51fa25ad>, line 17)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# 저장된 모델과 토크나이저 로드
model_dir = "/content/drive/MyDrive/삼성증권/Model_dic/Kakao-1"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForQuestionAnswering.from_pretrained(model_dir)

# QA 파이프라인 설정
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# 테스트 데이터 준비
test_df = pd.DataFrame(dataset['validation']).sample(n=10, random_state=42)  # 무작위로 10개 샘플 추출
test_dataset = Dataset.from_pandas(test_df)

# 데이터 전처리
def preprocess_test_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    return inputs

tokenized_test_dataset = test_dataset.map(preprocess_test_function, batched=True, remove_columns=test_dataset.column_names)

# 테스트 데이터로 예측 수행
for index, row in test_df.iterrows():
    context = row['context']
    question = row['question']
    answer = row['answers']['text'][0]

    # 모델 예측
    result = qa_pipeline(question=question, context=context)

    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Expected Answer: {answer}")
    print(f"Predicted Answer: {result['answer']}\n")


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Context: 2017년 2월 13일 오전 9시경 김정남은 말레이시아 세팡에 위치한 쿠알라룸푸르 국제공항에서 무언가를 맨손에 묻힌 여성 2명이 얼굴을 문지르고 얼마 후 사망하였다. 보툴리눔으로 죽었다는 설도 나왔고, 독극물 스프레이를 맞고 사망하였다는 설도 있다. 또한 일부는 독침으로 죽었다는 가설도 있다고 한다. 김정남 암살은 김정은이 권력을 승계받은 2011년 다음해 인 2012년 초 부터 진행되었으며, 정찰총국 등 북한 정보당국의 최순위 목표였다고 국가정보원이 밝혔다. 또한 국가정보원은 2012년 4월 김정남이 김정은 위원장에게 “살려달라”는 내용의 서신을 보낸 사실도 공개하였다. 2017년 2월 19일 말레이시아 경찰은 공식 브리핑을 통해 암살을 실행한 인도네시아,베트남 출신 여성 2명을 배후에서 사주한 일당 4명이 모두 조선민주주의인민공화국 국적이라고 발표하였다.
Question: 김정남을 살해한 배후와 사주한 일당의 국적은?
Expected Answer: 조선민주주의인민공화국
Predicted Answer: 조선민주주의인민공화국

Context: 1977년 입사하여 원액2과에서 7년간 근무하고 1983년 퇴사한 김봉환은 1990년 10월 30일 초진으로 이황화탄소 중독판정을 받고 회사에 산재요양신청을 냈으나 거부당했다. 1991년 1월 5일 김봉환은 직업병 증세인 정신분열로 인해 사망하였다. 김봉환이 활동하던 '원진직업병피해노동자협의회'(이하 원노협)는 이황화탄소 중독 여부를 검진 받지 못하고 사망한 데에 대한 책임을 물어 사업주를 처벌하고 부검 담당 검사에게 의뢰하여 원진직업병으로 인한 사망 여부를 판정해 줄 것을 노동부 측에 요구하였다. 그러나 병리학 검사를 의뢰받은 고려대학교의 비협조적인 태도와 회사와 사측 추천 의사들의 무성의로 사건을 제대로 마무리 짓지 못하였다. 유족과 원노협 회원들은 영결식을 위한 회사 출입을 봉쇄당하여 회사 정문 앞에서 시신투쟁을 벌이고 평일에는 대책위와 조합원을 중심으로 주말에는 수도권의 노동자와 사회단체, 학생들의

## Test_2

In [ ]:
# 임의의 문맥과 질문
context = """
ChatGPT는 OpenAI가 개발한 대규모 언어 모델입니다. 이 모델은 자연어 처리(NLP) 기술을 기반으로 하여, 다양한 언어 작업을 수행할 수 있습니다.
이를 통해 텍스트 생성, 번역, 요약, 감정 분석 등 여러 작업을 할 수 있습니다.
"""
question = "ChatGPT는 무엇을 기반으로 개발되었나요?"

# 모델 예측
result = qa_pipeline(question=question, context=context)

# 예측 결과 출력
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Predicted Answer: {result['answer']}")

Question: ChatGPT는 무엇을 기반으로 개발되었나요?
Context: 
ChatGPT는 OpenAI가 개발한 대규모 언어 모델입니다. 이 모델은 자연어 처리(NLP) 기술을 기반으로 하여, 다양한 언어 작업을 수행할 수 있습니다. 
이를 통해 텍스트 생성, 번역, 요약, 감정 분석 등 여러 작업을 할 수 있습니다.

Predicted Answer: 자연어 처리
